# Running Omnipose
Omnipose works with all of Cellpose. Use `omni=True` to enable it, even when using Cellpose-based models. This might help avoid over-segmentation without retraining. 

Before running this notebook, install the latest version of Omnipose from GitHub. 

In [ ]:
# make local editable packages automatically reload
%load_ext autoreload
%autoreload 2

# Import dependencies
import numpy as np
from cellpose import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# for plotting 
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300


### Load files
There are several ways to load your image files into a notebook. If you have a specific set of images, put their full paths into a list, for example:
```python
# make it a list even if there is only one file
files = ['path_to_image_1'] 
files = ['path_to_image_1','path_to_image_2']

# you can also add to the list like so:
files = files + ['path_to_image_3']
```

Alternatively, you can load all the images in a directory. Here are a few templates you can use to get the list of directories automatically by searching for image files matching a cetrain file lane with an extension and keywords in the file name. 
```python
from pathlib import Path

basedir = 'path_to_image_folder'
files = [str(p) for p in Path(basedir).rglob("*.tif")] #use rglob to search subfolders recusrively 
files = [str(p) for p in Path(basedir).glob("*C1.png")] # change the search string to grab only one channel
files = [str(p) for p in Path(basedir).glob("*488*.png")] # specify a match anywhere in the file name
```

We can also use the `cellpose.io` library to grab all the images in the test_files folder. Very handy for grabbing images of different extensions. Here we are using an RGB image related to the Omnipose bact_phase dataset and four images from the DeLTA 2.0 training set on which the bact_phase_omni model has never been trained.  

In [ ]:
from pathlib import Path
import os
from cellpose import io

basedir = os.path.join(Path.cwd().parent,'test_files')
files = io.get_image_files(basedir)
files

Read in the images from the file list. It's a good idea to display the images before proceeding. Here I happen to be reading in some RBG tiles of grayscale phase contrast images (such as you might use for figures etc.) as well as some single-channel images. 

In [ ]:
from cellpose import io, transforms 
imgs = [io.imread(f) for f in files]

# print some info about the images.
for i in imgs:
    print(i.shape)
nimg = len(imgs)
print(nimg)

plt.figure(figsize=[4]*2) # initialize figure
for k in range(len(imgs)):
    img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
    # if len(img.shape)>2:
        # imgs[k] = img[:,:,1] # could pick out a specific channel
        # imgs[k] = np.mean(img,axis=-1) # or just turn into grayscale 
        
    imgs[k] = transforms.normalize99(imgs[k],omni=True)
    plt.subplot(1,len(files),k+1)
    plt.imshow(imgs[k],cmap='gray')
    plt.axis('off')

### Initialize 
Here we use one of the built-in model names. You can print out the available model names, too:

In [ ]:
from cellpose import models
from cellpose.models import MODEL_NAMES
print(MODEL_NAMES)

We will choose the `bact_phase_omni` model. 

In [ ]:
model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

### Run segmentation 

In [ ]:
chans = [0,0] #this means segment based on first channel, no second channel 

n = [0] # make a list of integers to select which images you want to segment
n = range(nimg) # or just segment them all 

# define parameters
mask_threshold = -1 
verbose = 0 # turn on if you want to see more output 
use_gpu = use_GPU #defined above
transparency = True # transparency in flow output
rescale=None # give this a number if you need to upscale or downscale your images
omni = True # we can turn off Omnipose mask reconstruction, not advised 
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
resample = True #whether or not to run dynamics on rescaled grid or original grid 

masks, flows, styles = model.eval([imgs[i] for i in n],channels=chans,rescale=rescale,mask_threshold=mask_threshold,transparency=transparency,
                                  flow_threshold=flow_threshold,omni=omni,resample=resample,verbose=verbose)

### Plot the results

In [ ]:
from cellpose import plot
import omnipose

for idx,i in enumerate(n):

    maski = masks[idx]
    flowi = flows[idx][0]

    fig = plt.figure(figsize=(12,5))
    # plot.show_segmentation(fig, omnipose.utils.normalize99(imgs[i][:,:,0:2]), maski, flowi, channels=chans, omni=True, bg_color=0)
    plot.show_segmentation(fig, omnipose.utils.normalize99(imgs[i]), maski, flowi, channels=chans, omni=True, bg_color=0)
    
    plt.tight_layout()
    plt.show()

### Save the results

In [ ]:
io.save_masks(imgs, masks, flows, files, 
              tif=True, #whether to use PNG or TIF format
              suffix='_example', # suffix to add to files if needed 
              save_flows=True, 
              save_outlines=True, # save outline images 
              dir_above=0, # save output in the image directory or in the directory above (at the level of the image directory)
              in_folders=True, # save output in folders (recommended)
              save_txt=False, # txt file for outlines in imageJ
              save_ncolor=True) # save ncolor version of masks for visuaizatin and editing 

### Debug results

The RGB flows shown above will give you some insight as to if there is an issue with the flow field outputs, but you can also check out the boundary and distance output:

In [ ]:
for idx,i in enumerate(n):

    disti = flows[idx][2]
    bdi = flows[idx][4]
    fig = plt.figure(figsize=(12,5))
    plt.imshow(np.hstack([bdi,disti]))
    plt.axis('off')
    plt.tight_layout()
    plt.show()